In [17]:
import modelinference
import prompts
import importlib
import datetime
import torch

from accelerate import notebook_launcher

In [2]:
importlib.reload(modelinference)

<module 'modelinference' from '/project/modelinference.py'>

In [3]:
torch.cuda.device_count()

2

In [27]:

run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
    'model_s3_loc': 'deepseek14Q',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoT'],
    'multi-gpu':True,
    'dataset': 'data_annual_pit_spx',
    'data_location': 'data_annual_pit_spx.json'
}

run_config = {
    'model_hf_id': 'Qwen/Qwen2.5-7B-Instruct',
    'model_s3_loc': 'qwen',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

run_name = f"{run_config['model_s3_loc']} - {run_config['dataset']}"

In [6]:
def inference_multi_gpu():
    ir = modelinference.InferenceRun(run_name, run_config)
    ir.run_multi_gpu()

In [7]:
notebook_launcher(inference_multi_gpu(), num_processes=torch.cuda.device_count())

qwen


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Loading Data
Saving data...


Memory footprint: 15.2 GB
waiting...


TypeError: can only concatenate str (not "dict") to str

In [13]:
ir = modelinference.InferenceRun(run_name, run_config)

In [14]:
prompts = ir.create_all_prompts()

In [15]:
len(prompts)

3009

In [19]:
ir2 = modelinference.InferenceRun(run_name, run_config) #indu annual

In [22]:
p2 = ir2.create_all_prompts()

In [23]:
len(p2)

232

In [28]:
ir3 = modelinference.InferenceRun(run_name, run_config) #indu annual
p3 = ir3.create_all_prompts()
len(p3)

896

In [10]:
import json

In [12]:
with open(f'Data/{run_name}/prompts.json') as f:
    p = json.load(f)

In [16]:
p[0]['prompt']

[{'role': 'system',
  'content': {'date': '2025-02-10',
   'prompt': "You are a financial analyst tasked with analyzing the financial statements of a company to predict the direction of future earnings.Follow the steps below to perform the analysis. 1. Identify notable changes in the balance sheet and income statement. 2. Compute key financial ratios to understand the health of the company. State the formula before calculating. Compute profitability ratios, leverage ratios, liquidity ratios and efficiency ratios. 3. Interpret each of the ratios. 4. Predict the direction of future earnings in JSON format with a clear recommendation and size of the increase or decrease: {'earnings':'INCREASE', 'magnitude':'LARGE'} or {'earnings':'DECREASE','SMALL'} 5. Provide a rational in less than 250 words. Company Financial Statements: "}},
 {'role': 'user',
  'content': 'Income Statement:                                                        t           t-1           t-2           t-3           t-4

In [8]:
import utils.modelHelper as mh
import importlib
importlib.reload(mh)

<module 'utils.modelHelper' from '/project/utils/modelHelper.py'>

In [9]:
helper = mh.ModelHelper('tmp/fs')

In [10]:
helper.clear_folder('qwen')
helper.clear_folder(f'Data/{run_name}')

In [6]:
import os

In [7]:
print(os.)

NameError: name 'ROOT_DIR' is not defined

In [10]:
os.path.abspath(os.getcwd())

'/project'

In [ ]:
with open()